## Homework 2

In [39]:
import cv2
import colorsys

In [69]:
def draw_bounding_box(img, x, y, w, h):
    cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
    (tw, th), _ = cv2.getTextSize(' Object ', cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(img, (x, y - 30), (x + tw, y), (36,255,12), -1)
    cv2.putText(img, ' Object ', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
def draw_color_indicator(img, bgr, x, y):
    cv2.rectangle(img, (x, y), (x+30, y+30), bgr, -1)
    (tw, th), _ = cv2.getTextSize(' target color ', cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(img, (x + 30, y), (x + tw + 30, y + 30), (255, 255, 255), -1)
    cv2.putText(img, ' target color ', (x + 30, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

In [70]:
camera = cv2.VideoCapture(0)

cv2.namedWindow('Frame')

# 트랙바 선언
cv2.createTrackbar('Hue', 'Frame', 0, 179, lambda x: x)
cv2.createTrackbar('Saturation', 'Frame', 0, 255, lambda x: None)
cv2.createTrackbar('Value', 'Frame', 0, 255, lambda x: None)
cv2.createTrackbar('Threshold', 'Frame', 0, 255, lambda x: None)


while True:
    # grab the current frame
    (grabbed, frame) = camera.read()
    
    # 영상이 끝나면 종료
    if not grabbed:
        break

    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 추출하려는 색상 가져오기
    hue = cv2.getTrackbarPos('Hue','Frame')
    saturation = cv2.getTrackbarPos('Saturation','Frame')
    value = cv2.getTrackbarPos('Value','Frame')
    threshold = cv2.getTrackbarPos('Threshold','Frame')
    
    # 색상을 BGR 형식으로 변경
    rgb = colorsys.hsv_to_rgb(hue / 179., saturation / 255., value / 255.)
    rgb = tuple(map(lambda x: x * 255, rgb))
    bgr = (rgb[2], rgb[1], rgb[0])
    
    # 추출하려는 색상 표시
    draw_color_indicator(frame, bgr, 20, 20)
    
    # 추출하려는 색상의 하한과 상한 지정
    lower = (max(0, hue - threshold/2), max(0, saturation - threshold), max(0, value - threshold))
    upper = (min(179, hue + threshold/2), min(255, saturation + threshold), min(255, value + threshold))
    
    # construct a mask for all colors in the current HSV range, then
    # perform a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # find contours in the mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)[0]


    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use it to compute
        c = max(cnts, key=cv2.contourArea)
        
        # 바운딩 박스 생성
        x,y,w,h = cv2.boundingRect(c)

        if w > 6 and h > 6:
            draw_bounding_box(frame, x, y, w, h)
    

    # 이미지 출력
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(Hue@Frame): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(Saturation@Frame): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(Value@Frame): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/mo

-1